In [9]:
# Script to delete non-anomalous hits from .dat files after GLOBULAR clustering.

import numpy as np
import shutil
import psutil
import os

In [ ]:
hit_dats = np.load('/datax/scratch/benjb/C23_L_unique_dat_list.npy', allow_pickle=True)[:,1]

# Need to copy .dats into new folder so previous work isn't affected.
print('Copying dats to /datax/scratch/benjb/C23_L_dats_after_8_HDB_epochs/ ...')

for dat in np.unique(hit_dats):
    shutil.copy(dat, '/datax/scratch/benjb/C23_L_dats_after_8_HDB_epochs/')

print('Copied!')

In [3]:
iii = np.load('/datax/scratch/benjb/C23_L_non_anom_centroid_idxs.npy')
hit_params = np.load('/datax/scratch/benjb/C23_L_unique_param_array.npy', allow_pickle=True)
hit_dats = np.load('/datax/scratch/benjb/C23_L_unique_dat_list.npy', allow_pickle=True)[:,1]

In [4]:
freqs_for_deletion = hit_params[iii, 0]
dats_for_deletion = hit_dats[iii]

In [ ]:
total_count = len(dats_for_deletion)

# This can take a while.
print('Beginning hit deletion ...')

for i in range(len(dats_for_deletion)):
    freq = freqs_for_deletion[i]
    dat_path = '/datax/scratch/benjb/C23_L_dats_after_8_HDB_epochs/'+os.path.basename(dats_for_deletion[i])
    if i%20000 == 0:
        print(f'{i} of {total_count}: Removing hit at {freq} MHz from {dat_path}')

    bytes_available = psutil.virtual_memory()[1]
    if bytes_available <= 32e9:
        print(f'Memory dangerously low: {bytes_available} bytes remaining. Breaking ...')
        break

    lines = []
    for line in open(dat_path):
        if not str(freq) in line:
            lines.append(line)

    with open(dat_path, 'w') as file:
        file.writelines(lines)
        file.close()